In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [8]:
import requests
page= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page,'lxml')

In [9]:
#get the table
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

df = pd.DataFrame(data = data,columns = columns)

In [12]:
#Remove'Not assigned' Boroughs
df = df[df['Borough'] != 'Not assigned']
#remove duplicates
df = df.drop_duplicates()
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighbourhood'].replace("Not assigned", df["Borough"],inplace=True)

In [13]:
geo="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(geo)

geo_data.columns=['Postal Code', 'Latitude', 'Longitude']

new_df= pd.merge(df, geo_data, how='inner', on="Postal Code")
new_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [14]:
#only use the borough in North York
borough_ny_df=new_df[new_df["Borough"].str.contains("North York")].reset_index(drop=True)
borough_ny_df.size

120

In [15]:
borough_ny_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [16]:
borough_ny_df.shape

(24, 5)

In [17]:

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import geopy
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [18]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
# create map of Torronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

In [21]:
#only use the borough with word North York
borough_ny_df=new_df[new_df["Borough"].str.contains("North York")].reset_index(drop=True)
borough_ny_df.size

120

In [22]:
borough_ny_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [23]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'SS50NHLBCQAJRDC24PAEEG1DNSFUZSIVTSD35F3LP3LQVWST' 
CLIENT_SECRET = 'UMPFTH40H3FU3ZBQ4ZOHIP0BTOZ5D1KB11QUHFGHD5QADBBB' 
VERSION = '20200721' # Foursquare API version
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SS50NHLBCQAJRDC24PAEEG1DNSFUZSIVTSD35F3LP3LQVWST
CLIENT_SECRET:UMPFTH40H3FU3ZBQ4ZOHIP0BTOZ5D1KB11QUHFGHD5QADBBB


In [148]:
# method to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [149]:
ny_venues_df = getNearbyVenues(borough_ny_df['Neighbourhood'],borough_ny_df['Latitude'],borough_ny_df['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [150]:
ny_venues_df.shape

(248, 7)

In [151]:
ny_venues_df

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Roots,43.718214,-79.463893,Boutique
8,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),43.719096,-79.462675,Furniture / Home Store
9,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant


In [152]:
print('There are {} uniques categories.'.format(len(ny_venues_df['Venue Category'].unique())))

There are 106 uniques categories.


In [153]:
# write venue category to int mapping function

def sameShopToInt(categorieString):
    categorieText = categorieString.lower()
    if 'coffee' in categorieText or 'café' in categorieText or 'tea' in categorieText:
        return 1
    return 0

def coopToInt(categorieString):
    categorieText = categorieString.lower()
    if 'shopping' in categorieText or 'movie' in categorieText or 'thai' in categorieText or 'vietnamese' in categorieText or 'cantonese' in categorieText or 'korean' in categorieText or 'sushi' in categorieText or 'chinese' in categorieText:
        return 1
    return 0

In [154]:
ny_venues_df['CompetitionNumber'] = ny_venues_df['Venue Category'].apply(sameShopToInt)
ny_venues_df['CooperationNumber'] = ny_venues_df['Venue Category'].apply(coopToInt)
venuesCleaned = ny_venues_df[(ny_venues_df['CompetitionNumber'] == 0) & (ny_venues_df ['CooperationNumber']==1)]
print(venuesCleaned.shape)
venuesCleaned

(16, 9)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,CompetitionNumber,CooperationNumber
9,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant,0,1
45,Don Mills,43.725900,-79.340923,Congee Star 帝王名粥,43.726586,-79.341833,Chinese Restaurant,0,1
61,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,Wakame Sushi,43.755382,-79.440945,Sushi Restaurant,0,1
67,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,Sheppard Plaza,43.755695,-79.439613,Shopping Mall,0,1
74,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,SilverCity,43.778681,-79.344085,Movie Theater,0,1
75,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777994,-79.343665,Shopping Mall,0,1
132,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Manchu Wok,43.778225,-79.343302,Chinese Restaurant,0,1
150,Bayview Village,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant,0,1
160,Downsview,43.739015,-79.506944,jane sheppard mall,43.740104,-79.512552,Shopping Mall,0,1
176,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Satay on the Road,43.735310,-79.419783,Thai Restaurant,0,1


In [183]:
#sum up restaurant and sport number for each neighboorhood

insertComp = []
insertCoop = []
for nbh in borough_ny_df['Neighbourhood']:
    new = venuesCleaned[venuesCleaned.Neighbourhood == nbh]
    Comp = new['CompetitionNumber'].sum()
    Coop = new['CooperationNumber'].sum()
    insertComp.append(Comp)
    insertCoop.append(Coop)
borough_ny_df['CompetitionNumber'] = insertComp
borough_ny_df['CooperationNumber'] = insertCoop
print('Shape: '+str(borough_ny_df.shape))
borough_ny_df.dropna(inplace = True)
print('After drop na Values: '+str(borough_ny_df.shape))
borough_ny_df

Shape: (24, 8)
After drop na Values: (24, 8)


,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude,CompetitionNumber,CooperationNumber
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,0,0
1,0,M4A,North York,Victoria Village,43.725882,-79.315572,0,0
2,2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,1
3,2,M3B,North York,Don Mills,43.745906,-79.352188,0,1
4,0,M6B,North York,Glencairn,43.709577,-79.445073,0,0
5,2,M3C,North York,Don Mills,43.725900,-79.340923,0,1
6,0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,0
7,3,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,2
8,4,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,3
9,0,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,0,0


In [161]:
# set number of clusters
kclusters = 5

df_cleaned = borough_ny_df.drop(['Postal Code','Borough','Neighbourhood','Latitude','Longitude'], axis = 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_cleaned)

kmeans.labels_[0:10]

array([0, 0, 2, 2, 0, 2, 0, 3, 4, 0], dtype=int32)

In [162]:
borough_ny_df.insert(0, 'Cluster Labels', kmeans.labels_)
borough_ny_df.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude,CompetitionNumber,CooperationNumber
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,0,0
1,0,M4A,North York,Victoria Village,43.725882,-79.315572,0,0
2,2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,1
3,2,M3B,North York,Don Mills,43.745906,-79.352188,0,1
4,0,M6B,North York,Glencairn,43.709577,-79.445073,0,0


In [164]:
import matplotlib.cm as cm
import matplotlib.colors as colors


latitude = 43.706397
longitude = -79.309937

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(borough_ny_df['Latitude'], borough_ny_df['Longitude'], borough_ny_df['Neighbourhood'], borough_ny_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [173]:
#the first cluster
borough_ny_df.loc[borough_ny_df['Cluster Labels'] == 0, borough_ny_df.columns[[1] + list(range(4, borough_ny_df.shape[1]))]]

,Postal Code,Latitude,Longitude,CompetitionNumber,CooperationNumber
0,M3A,43.753259,-79.329656,0,0
1,M4A,43.725882,-79.315572,0,0
4,M6B,43.709577,-79.445073,0,0
6,M2H,43.803762,-79.363452,0,0
9,M3J,43.767980,-79.487262,0,0
12,M2L,43.757490,-79.374714,0,0
14,M6L,43.713756,-79.490074,0,0
15,M9L,43.756303,-79.565963,0,0
16,M2M,43.789053,-79.408493,0,0
19,M9M,43.724766,-79.532242,0,0


In [174]:
#the second cluster
borough_ny_df.loc[borough_ny_df['Cluster Labels'] == 1, borough_ny_df.columns[[1] + list(range(4, borough_ny_df.shape[1]))]]

,Postal Code,Latitude,Longitude,CompetitionNumber,CooperationNumber
20,M2N,43.77012,-79.408493,0,5


In [175]:
#the third cluster
borough_ny_df.loc[borough_ny_df['Cluster Labels'] == 2, borough_ny_df.columns[[1] + list(range(4, borough_ny_df.shape[1]))]]

,Postal Code,Latitude,Longitude,CompetitionNumber,CooperationNumber
2,M6A,43.718518,-79.464763,0,1
3,M3B,43.745906,-79.352188,0,1
5,M3C,43.725900,-79.340923,0,1
10,M2K,43.786947,-79.385975,0,1
11,M3K,43.737473,-79.464763,0,1
13,M3L,43.739015,-79.506944,0,1
17,M3M,43.728496,-79.495697,0,1
21,M3N,43.761631,-79.520999,0,1


In [176]:
#the fourth cluster
borough_ny_df.loc[borough_ny_df['Cluster Labels'] == 3, borough_ny_df.columns[[1] + list(range(4, borough_ny_df.shape[1]))]]

,Postal Code,Latitude,Longitude,CompetitionNumber,CooperationNumber
7,M3H,43.754328,-79.442259,0,2
18,M5M,43.733283,-79.419750,0,2


In [179]:
#the fifth cluster
borough_ny_df.loc[borough_ny_df['Cluster Labels'] == 4, borough_ny_df.columns[[1] + list(range(4, borough_ny_df.shape[1]))]]

,Postal Code,Latitude,Longitude,CompetitionNumber,CooperationNumber
8,M2J,43.778517,-79.346556,0,3


In [ ]:
#With the maximum cooperation number of 5 and minimum competition number of 0, the second cluster will be the best cluster
#Therefore, the optimal location to open a bubble tea shop is M2N at (43.77012,79.408493). 